# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Packages

In [1]:
from __future__ import division
import sklearn
import pandas as pd 
import numpy as np 
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv("amazon_baby.csv", encoding="utf8")

Now, let us see a preview of what the dataset looks like.

In [3]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3.0
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5.0
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5.0
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5.0
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
#products[269:270]
products.loc[[269]]

,name,review,rating
269,The First Years Massaging Action Teether,A favorite in our house!,5.0


**IMPORTANT**. Make sure to fill n/a values in the review column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the review columns as follows:

In [5]:
products = products.fillna({'review':''})  # fill in N/A's in the review column
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3.0
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5.0
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5.0
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5.0
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5.0


Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. You can research more about a smart handling of punctuations. **Keyword: Tokenization**

In [6]:
def remove_punctuation(text):
    import string
    return str(text).translate(str.maketrans('','',string.punctuation)) 

products['review_clean'] = products['review'].apply(remove_punctuation)

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

101477

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5.0,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5.0,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5.0,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5.0,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5.0,When the Binky Fairy came to our house we didn...,1
...,...,...,...,...,...
111680,Cloud B Gentle Giraffe Soother &amp; Plush Rat...,The baby adores it and the parents love it eve...,5.0,The baby adores it and the parents love it eve...,1
111681,"NoJo Watch Me Grow 4 Piece Set, Plum/Pink",Beautiful Set! I love this set. It has great c...,5.0,Beautiful Set I love this set It has great col...,1
111683,"Graco Vie4 Stroller, Olivia",Let me preface this with: my baby isn't here y...,5.0,Let me preface this with my baby isnt here yet...,1
111684,Chicco Cortina Together Double Stroller - Cubes,"I love the other Chicco products that I own, s...",1.0,I love the other Chicco products that I own so...,-1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `random_state=42` so that everyone gets the same result.

In [9]:
from sklearn.model_selection import train_test_split
X = products['review_clean'].to_numpy()
y = products['sentiment'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train.shape)
print(X_test.shape)

(67989,)
(33488,)


# Build the word count vector for each review

6. We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.

Compute the occurrences of the words in each review and collect them into a row vector.

Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.

Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(X_train)
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(X_test)
print(test_matrix.shape)

(33488, 78340)


Keep in mind that the test data must be transformed in the same way as the training data.



# Train a sentiment classifier with logistic regression

7. Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

**Note:** This line may take 1-2 minutes.

In [11]:
# Use sklearn's LogisticRegression
# Your code here 
from sklearn.linear_model import LogisticRegression

In [12]:
LogR = LogisticRegression()
LogR.fit(train_matrix, y_train)
sentiment_model = LogR.coef_


/home/sangdh/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
print(sentiment_model.shape)

(1, 78340)


8. There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

**Quiz Question:** How many weights are >= 0?

In [14]:
# Your code here 
positive_weights = [i for i in sentiment_model[0] if i>=0]
positive_weights = np.array(positive_weights)
print(len(positive_weights))

56532


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **X_sample_test_data**.

In [15]:
X_sample_test_data = X_test[10:13]
y_sample_test_data = y_test[10:13]
print(X_sample_test_data)
print(y_sample_test_data)

['I bought this for my grandson  It has been one of his favorite toys and teethers  Im sure it will be a favorite of his new brother too'
 'i bought this pump from a locally owned store the owner was my doulaIt seemed ok at first but the suction was kind of weak When I first had my daughter I had a hospital grade pump on loan because of my babys weight drop I tried again to use it even though the suction was seak and the milk kept going up the tube even though I used it properly It started to make the pump break because it went right up the tube and in to the pump itself When i polled it apart as much as i could I found a spungey part inside that was saturated in wet milk and it seemed like it was growing something because it must not have been the first time my milk went up the tube My doula has a call in to the company right now but no resalution has been worked out yet but I prefer pumping by hand over using this even when it did work properly It just didnt have enough guts to itI l

Let's dig deeper into the second row of the **X_sample_test_data**. Here's the full review:

In [16]:
X_sample_test_data[1]

'i bought this pump from a locally owned store the owner was my doulaIt seemed ok at first but the suction was kind of weak When I first had my daughter I had a hospital grade pump on loan because of my babys weight drop I tried again to use it even though the suction was seak and the milk kept going up the tube even though I used it properly It started to make the pump break because it went right up the tube and in to the pump itself When i polled it apart as much as i could I found a spungey part inside that was saturated in wet milk and it seemed like it was growing something because it must not have been the first time my milk went up the tube My doula has a call in to the company right now but no resalution has been worked out yet but I prefer pumping by hand over using this even when it did work properly It just didnt have enough guts to itI like the idea of the small 1 cup design and I think the company has the no how to make this a better product but until there are allot of ch

That review seems pretty positive.

Now, let's see what the first row of the **X_sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [17]:
X_sample_test_data[0]

'I bought this for my grandson  It has been one of his favorite toys and teethers  Im sure it will be a favorite of his new brother too'

We will now make a **class** prediction for the **X_sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  
We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in sample_test_data. In scikit-learn, you can call the decision_function() function.

**Hint**: You'd probably need to convert sample_test_data into the sparse matrix format first.

In [18]:
sample_test_matrix = vectorizer.transform(X_sample_test_data)
scores = LogR.decision_function(sample_test_matrix)
scores=np.array(scores)
print(scores)

[ 5.10258712 -7.48315327  9.94852971]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [19]:
y_predicted = [1 if i>0 else -1 for i in scores]
print(y_predicted)

[1, -1, 1]


**Checkpoint**: Make sure your class predictions match with the ones obtained from sentiment_model. The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

In [20]:
print("Class predictions according to GraphLab Create:")
print(LogR.predict(sample_test_matrix))

Class predictions according to GraphLab Create:
[ 1 -1  1]


### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [21]:
def sigmoid(z):
    return 1.0/(1+np.exp(-z))

In [22]:
p = sigmoid(scores)
p.shape

(3,)

**Checkpoint**: Make sure your probability predictions match the ones obtained from sentiment_model.

In [23]:
LogR.predict_proba(sample_test_matrix)


array([[6.04423893e-03, 9.93955761e-01],
       [9.99437835e-01, 5.62164753e-04],
       [4.77955745e-05, 9.99952204e-01]])

**Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

13. We now turn to examining the full test dataset, test_data, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the sentiment_model, find the 20 reviews in the entire test_data with the highest probability of being classified as a positive review. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:

1. Make probability predictions on test_data using the sentiment_model.
2. Sort the data according to those predictions and pick the top 20.

In [24]:
X_test_matrix = vectorizer.transform(X_test)
Score_total = LogR.predict_proba(X_test_matrix)
print(Score_total.shape)


(33488, 2)


In [25]:
index_20_top = np.argsort(Score_total[:,1])[0:20]
#print(index_20_top)
print(Score_total[index_20_top])

[[1.00000000e+00 8.82188576e-19]
 [1.00000000e+00 1.02534246e-15]
 [1.00000000e+00 5.27717802e-14]
 [1.00000000e+00 6.82950688e-14]
 [1.00000000e+00 1.19476841e-13]
 [1.00000000e+00 2.21186015e-12]
 [1.00000000e+00 2.71373295e-12]
 [1.00000000e+00 3.12534019e-12]
 [1.00000000e+00 3.86254617e-12]
 [1.00000000e+00 9.50389701e-12]
 [1.00000000e+00 1.22230181e-11]
 [1.00000000e+00 1.53300380e-11]
 [1.00000000e+00 1.62210375e-11]
 [1.00000000e+00 3.02314373e-11]
 [1.00000000e+00 6.24508694e-11]
 [1.00000000e+00 9.02407764e-11]
 [1.00000000e+00 9.85586432e-11]
 [1.00000000e+00 1.22164273e-10]
 [1.00000000e+00 2.87630547e-10]
 [1.00000000e+00 4.05900170e-10]]


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [26]:
index_20_of_most_negative_reviews = np.argsort(Score_total[:,0])[0:20]
print(Score_total[index_20_of_most_negative_reviews])

[[0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00]
 [2.22044605e-16 1.00000000e+00]
 [2.22044605e-16 1.00000000e+00]
 [2.22044605e-16 1.00000000e+00]
 [4.44089210e-16 1.00000000e+00]
 [6.66133815e-16 1.00000000e+00]
 [8.88178420e-16 1.00000000e+00]
 [3.77475828e-15 1.00000000e+00]
 [4.44089210e-15 1.00000000e+00]
 [4.88498131e-15 1.00000000e+00]
 [5.32907052e-15 1.00000000e+00]
 [5.32907052e-15 1.00000000e+00]
 [6.43929354e-15 1.00000000e+00]]


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [27]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    ...
    pre = model.predict(data)
    total_examples = np.shape(true_labels)
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    ...
    correctly_classified_examples = np.sum(pre==true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    ...
    accuracy = correctly_classified_examples/total_examples
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [28]:
get_classification_accuracy(LogR, X_test_matrix, y_test)

array([0.92686933])

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [29]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [30]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [31]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(X_train)
test_matrix_word_subset = vectorizer_word_subset.transform(X_test)

In [32]:
print(vectorizer_word_subset)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None,
                vocabulary=['love', 'great', 'easy', 'old', 'little', 'perfect',
                            'loves', 'well', 'able', 'car', 'broke', 'less',
                            'even', 'waste', 'disappointed', 'work', 'product',
                            'money', 'would', 'return'])


## Train a logistic regression model on a subset of data

Now build a logistic regression classifier with train_matrix_word_subset as features and sentiment as the target. Call this model **simple_model**.

In [33]:
# Your code here 
from sklearn.linear_model import LogisticRegression
simple_model=LogisticRegression()
simple_model.fit(train_matrix_word_subset,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [34]:
# Your code here 
accuracy = get_classification_accuracy(simple_model,test_matrix_word_subset,y_test)
print(accuracy)

[0.86174152]


Now, we will inspect the weights (coefficients) of the **simple_model**:

In [35]:
# Your code here 
the_weights = simple_model.coef_
print(the_weights.shape)

(1, 20)


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [36]:
# Your code here
np.sort(the_weights[0])[::-1]
print(the_weights)

[[ 1.36579684  0.92037179  1.23281615  0.14872827  0.50323904  1.44556512
   1.66148209  0.41657816  0.22728168  0.04267154 -1.7161958  -0.16801529
  -0.5110314  -1.92725218 -2.23007307 -0.55407425 -0.27982127 -0.88794598
  -0.33173491 -2.05583494]]


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [37]:
# Your code here 

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [38]:
# Your code here 

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [39]:
# Your code here 
accuracy_of_sentiment_model = get_classification_accuracy(LogR,train_matrix,y_train)
print(accuracy_of_sentiment_model)

[0.95372781]


Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [40]:
# Your code here 
accuracy_of_simple_model = get_classification_accuracy(simple_model,train_matrix_word_subset,y_train)
print(accuracy_of_simple_model)

[0.86119813]


**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [41]:
# Your code here 
get_classification_accuracy(
    LogR, 
    test_matrix, 
    y_test)

array([0.92686933])

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [42]:
# Your code here 
get_classification_accuracy(
    simple_model, 
    test_matrix_word_subset, 
    y_test)

array([0.86174152])

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [43]:
num_positive  = np.sum(np.array(y_train) == 1)
num_negative = np.sum(np.array(y_train) == -1)
print(num_positive)
print(num_negative)

56715
11274


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [44]:
# Your code here 
num_positive  = np.sum(np.array(y_test) == 1)
num_negative = np.sum(np.array(y_test) == -1)
print(np.round(num_positive/y_test.shape[0],2))

0.84


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

In [45]:
# Your code here 